# LLM tooling for Chemistry

<p style="font-style: italic; font-size: 18px; color: dimgray;">ChatGPT and Gemini</p>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2025

-----

## Introduction

This notebook shows how to do utilize Large Language Model (LLM) [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) with the Raku package ["LLM::Functions"](https://github.com/antononcube/Raku-LLM-Functions), [AAp1].

Non-trivial Stoichiometry examples are demonstrated with tools of the Raku package ["Chemistry::Stoichiometry"](https://raku.land/zef:antononcube/Chemistry::Stoichiometry), [AAp4].

At this point "LLM::Functions" supports function calling in the styles of OpenAI's ChatGPT and Google's Gemini. If the LLM configuration is not using ChatGPT or Gemini, the function calling style used is that of ChatGPT. (Many LLM providers -- other than OpenAI and Gemini -- tend to adhere to OpenAI's API.)

-----

## Setup

Load packages:

In [52]:
use JSON::Fast;
use LLM::Functions;
use LLM::Tooling;
use Chemistry::Stoichiometry;


Define LLM access configurations:

In [53]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', :8192max-tokens, temperature => 0.4);
sink my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash', :8192max-tokens, temperature => 0.4);

### JavaScript::D3

In [54]:
#%javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [55]:
#%js
js-d3-list-line-plot(10.rand xx 30, background => 'none')

----

## Diagram

Here is a [Mermaid-JS](https://mermaid.js.org) diagram that shows _single-pass_ LLM-and-tool interaction:


```mermaid
sequenceDiagram
    participant Developer
    participant Model as LLM Service (Gemini)
    participant Tool as External Function

    Note over Developer, Model: 1. Tool Definitions + Messages
    Developer->>Model: molecular-mass(molecule) <br> "What's the mass of SO2?"

    Note over Model: 2. Tool Calls
    Model->>Tool: molecular-mass("SO2")

    Note over Tool: 3. Execute Function Code
    Tool->>Model: {"mass": 64.058}

    Note over Model: 4. Results
    Model->>Developer: {"mass": 64.058}

    Note over Developer: 5. Final Response
    Developer->>Model: The molecular mass of SO2 is 64.058.
```


**Remark:** Instead of one invocation of a local tool -- as shown in the diagram -- often enough multiple LLM-&-local-computer interactions are used for LLM-tool computation.


---

## Chemistry computations examples

The package ["Chemistry::Stoichiometry"](https://raku.land/zef:antononcube/Chemistry::Stoichiometry), [AAp4], provides element data, a grammar (or parser) for chemical formulas, and subs for computing molecular masses and balancing equations. Here is an example of calling `molecular-mass`:

In [56]:
molecular-mass("SO2")

64.058

Balance chemical equation:

In [57]:
'Al + O2 -> Al2O3'
==> balance-chemical-equation

[4*Al + 3*O2 -> 2*Al2O3]

In [58]:
text-list-plot(random-real(100,10))

+---+----------+-----------+----------+-----------+--------+       
|                                                          |       
|   *    *                                                 |       
+                                                          +  80.00
|                                                          |       
|              *                                           |       
|                                                          |       
+                                                          +  60.00
|                          *                               |       
|                               *                      *   |       
+                                                          +  40.00
|                                           *              |       
|                    *                *           *        |       
|                                                          |       
+---+----------+-----------+----------+---------

----

## Define package functions as tools

Define a few subs -- _tools_ -- with sub- and argument descriptions (i.e. attached Pod values, or [declarator blocks](https://docs.raku.org/language/pod#Declarator_blocks)):

In [59]:
sink my @tool-objects =
        LLM::Tool.new(&molecular-mass),
        LLM::Tool.new(&balance-chemical-equation)
        ;

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


-----

## LLM-tool invocation example **1**

A single prompt or an array of prompts:

In [60]:
sink my $input = "What are the masses of SO2, O3, and C2H5OH? Also balance: C2H5OH + O2 = H2O + CO2."

LLM result:

In [ ]:
my $conf = llm-configuration($conf41-mini, tools => @tool-objects);

llm-synthesize(
        [$input, llm-prompt('NothingElse')('JSON')],
        e => $conf, 
        form => sub-parser('JSON'):drop)

**Remark:** It order to see the LLM-tool interaction use the Boolean option (adverb) `:echo` of `llm-synthesize`.

---

## LLM-tool invocation example 2

Let us add a plotting tool. We can use `text-list-plot` of ["Text::Plot"](https://raku.land/zef:antononcube/Text::Plot), but automatically derived tool spec -- via the sub `llm-tool-definition` used by `LLM::Tool` -- is somewhat incomplete. Here is auto-result:

In [ ]:
llm-tool-definition(&text-list-plot)

We could overwrite the tool-spec for `text-list-plot` with this spec:

```raku
my $spec = q:to/END/;
{
  "type": "function",
  "function": {
    "name": "text-list-plot",
    "description": "Make a string that represents a list-plot of the given arguments.",
    "parameters": {
      "type": "object",
      "properties": {
        "$x": {
          "type": "array",
          "description": "A list of a list of x-coordinates or x-labels",
          "items": {
            "anyOf": [
              { "type": "string" },
              { "type": "number" }
            ]
          }
        }
        "$y": {
          "type": "array",
          "description": "A list of y-coordinates",
          "items": {
            "type": "number"
          }
        }
      },
      "required": ["$x", "$y"]
    }
  }
}
END

my $t = LLM::Tool.new(&text-list-plot);
$t.json-spec = $spec;
```


Though, it is easier and more robust to define a new function that delegates to `text-list-plot`, and does some additional input processing that anticipates LLM derived argument values:

In [ ]:
"[64.058, 47.997, 48.61, 46.069]".split(/<[\[\],]>/, :skip-empty)».trim.raku

In [33]:
#| Make a string that represents a list-plot of the given arguments.
my sub list-plot(
    Str:D $x,             #= A list of comma separated x-coordinates or x-labels
    Str:D $y,             #= A list of comma separated y-coordinates
    Str:D :$x-label = '', #= Label of the x-axis
    Str:D :$y-label = '', #= Label of the y-axis
    ) {
    my @x = $x.split(/<[\[\],"]>/, :skip-empty)».trim.grep(*.chars);
    my @y = $y.split(/<[\[\],"]>/, :skip-empty)».trim».Num;
    
    # text-list-plot($x, $y, :$x-label, :$y-label, title => (@x.kv.map(-> $k, $v { "{$k+1} : $v" }).join(' | ') ))
    
    my @points = (1 ... @x.elems)».Num Z @y;

    text-list-plot(@points, :$x-label, :$y-label, point-char => @x )
}

&list-plot

In [132]:
#| Make a string that represents a list-plot of the given arguments.
my sub list-plot(
    Str:D $x,             #= A list of comma separated x-coordinates or x-labels
    Str:D $y,             #= A list of comma separated y-coordinates
    Str:D :$x-label = '', #= Label of the x-axis
    Str:D :$y-label = '', #= Label of the y-axis
    Str:D :$title = '',   #= Plot title
    ) {
  
    my @x = $x.split(/<[\[\],"]>/, :skip-empty)».trim.grep(*.chars);
    my @y = $y.split(/<[\[\],"]>/, :skip-empty)».trim».Num;
  
    # text-list-plot($x, $y, :$x-label, :$y-label, title => (@x.kv.map(-> $k, $v { "{$k+1} : $v" }).join(' | ') ))
    
    #my @points = (^@x.elems)».Num Z @y;
    #@points .= map({ %( x => $_.head, y => $_.tail, group => @x[$_.head] ) });
    #js-d3-list-plot(@points, :$x-label, :$y-label, :tooltip, title-color => 'Gray', background => '#1F1F1F', :legends, :grid-lines, :13point-size)
    my @points = (@x Z @y).map({ %( variable => $_.head, value => $_.tail ) });
    js-d3-bar-chart(@points, :$x-label, :$y-label, title-color => 'Gray', background => '#1F1F1F', :grid-lines)
}

&list-plot

In [133]:
#%js
list-plot("[\"SO2\", \"O3\", \"Mg2\", \"C2H5OH\"]","64.058, 47.997, 48.61, 46.069", y-label=>'mass')

Here we add the new tool to the tool list above:

In [134]:
sink my @tool-objects =
        LLM::Tool.new(&molecular-mass),
        LLM::Tool.new(&balance-chemical-equation)
        ;
        
sink @tool-objects.push(LLM::Tool.new(&list-plot))

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


In [142]:
my $input = q:to/END/;
What are the masses of SO2, O3, Mg2, and C2H5OH? 
Make a plot the obtained quantities: x-axes for the molecules, y-axis for the masses.
The plot has to have appropriate title and axes labels.
Return a JSON dictionary with keys "masses" and "plot".
END

my $conf = llm-configuration($conf41-mini, tools => @tool-objects);

my $res = llm-synthesize([
        $input, 
        llm-prompt('NothingElse')('JSON')
    ], 
    e => $conf,
    form => sub-parser('JSON'):drop
);

deduce-type($res)

Struct([masses, plot], [Hash, Str])

Here are the molecule masses:

In [143]:
$res<masses>

{C2H5OH => 46.069, Mg2 => 48.61, O3 => 47.997, SO2 => 64.058}

Here is the corresponding plot:

In [144]:
#%js
$res<plot>

--- 

## "Thoughtful" response

Here is a very informative, "thoughtful" response for a quantitative Chemistry question:

In [ ]:
#% markdown
my $input = "How many molecules a kilogram of water has? Use LaTeX for the formulas. (If any.)";

llm-synthesize($input, e => $conf)
==> { .subst(/'\[' | '\]'/, '$$', :g).subst(/'\(' | '\)'/, '$', :g) }() # Make sure LaTeX code has proper fences

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[LLM::Functions, Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::OpenAI, Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[WWW::Gemini, Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[Chemistry::Stoichiometry, Raku package](https://github.com/antononcube/Raku-Chemistry-Stoichiometry),
(2021-2025),
[GitHub/antononcube](https://github.com/antononcube).